In [16]:
import requests
import pandas as pd
import os
import matplotlib.pyplot as plt

# Set the API key
HEADERS = {
        "Accept": "application/json",
        "Authorization": "fsq3sBRRp8FffG0vdAxZKDQyxmSiK9QoQUhBBYoVNItiqjw=" 
    }

def extract_foursquare_data(df, headers=HEADERS):
    """
    Extracts data from the Foursquare API using the provided DataFrame and headers. The DataFrame should
    contain 'latitude' and 'longitude' columns. The function returns a new DataFrame with the extracted 
    data.

    :param df: DataFrame containing 'latitude' and 'longitude' columns
    :param headers: API headers for the Foursquare API
    :returns: DataFrame with the extracted Foursquare API data
    """

    foursquare_data_extract = []

    # Loop through the DataFrame rows
    for _, row in df[:10].iterrows():
        lat, lng = row['latitude'], row['longitude']
        url = f"https://api.foursquare.com/v3/places/nearby?ll={lat},{lng}"

        try:
            response = requests.get(url, headers=headers)
            response.raise_for_status()
        except requests.exceptions.RequestException as err:
            print(f"An error occurred: {err}")
            continue

        foursquare_data = response.json()

        for result in foursquare_data['results']:
            extracted_data = {
                "fsq_id" : result['fsq_id'],
                "category_id" : result['categories'][0]['id'],
                "category_name" : result['categories'][0]['name'],
                "chains" : result['chains'],
                "distance" : result['distance'],
                "latitude" : result['geocodes']['main']['latitude'],
                "longitude" : result['geocodes']['main']['longitude'],  
                "location_country" : result.get('location', {}).get('country'),
                "location_cross_street" : result.get('location', {}).get('cross_street'),
                "location_formatted_address" : result['location']['formatted_address'],  
                "location_locality" : result.get('location', {}).get('locality'),
                "location_postcode" : result.get('location', {}).get('postcode'),  
                "location_region" : result.get('location', {}).get('region'),   
                "location_timezone" : result['timezone'],   
                "name" : result['name']
            }
            foursquare_data_extract.append(extracted_data)

    # Convert the list of dictionaries to a DataFrame
    df2 = pd.DataFrame(foursquare_data_extract)

    return df2


In [17]:
def load_and_extract_data(csv_file_path, extraction_func, HEADERS):
    """
    Loads a CSV file into a DataFrame and extracts data using the provided extraction function.
    
    :param csv_file_path: Path to the CSV file to load
    :param extraction_func: Function for extracting data from the DataFrame
    :param headers: Headers to use for the API request
    :returns: DataFrame with the extracted data
    """
    df = pd.read_csv(csv_file_path)
    extracted_df = extraction_func(df, HEADERS)
    return extracted_df

# Path to the citybikes CSV file
csv_file_path = 'G:/Data/Python Project/citybike.csv'

# Call the function on the citybike dataframe to pull the foursquare API results
foursquare_df = load_and_extract_data(csv_file_path, extract_foursquare_data, HEADERS)


In [18]:
foursquare_df.to_csv('G:/Data/Python Project/foursquare.csv') 

In [13]:
print(foursquare_df)

                      fsq_id  category_id      category_name  \
0   4b65b9f1f964a520fefb2ae3        13031       Burger Joint   
1   4d0cef321f6bf04d75707a31        16037         Playground   
2   4deb8ba688774880e3387c0c        17065     Farmers Market   
3   4fdbc663e4b029940a58c201        16049          Surf Spot   
4   4be3345b660ec928226dcb3b        16017             Garden   
..                       ...          ...                ...   
86  570ff104498eeab32a85da3e        18021     Gym and Studio   
87  4c8bc4cf5e38b1f773f22af9        11073                Spa   
88  4b04191df964a5205c5122e3        11045               Bank   
89  618ca101ab4b8264f40b147a        13148  French Restaurant   
90  56d5a00fcd109c07f4c4afa5        13034               Café   

                                               chains  distance   latitude  \
0   [{'id': 'ab4a3530-d68a-012e-5619-003048cad9da'...        62  43.665065   
1                                                  []        63  43.664679 

In [20]:
import os
import requests
import json
import pandas as pd

business_id = 'srJ-hGeN_7H5oGT4wYz7uw'
api_key = "w7GSO5abCgHWgnZ-Vd21P00zNIY_nx-1ygAPL-5XngiWsGeBAPuTaMVPSB4ulo3veq0cjeEiPtM9IzErrIWwwrI3Wt5ELsCrx5qT4xDp6UgHBe83PU7LebN4MRx-ZHYx"
headers = {'Authorization': 'Bearer %s' % api_key}

def extract_yelp_data(df1, headers):
    """
    Extracts data from the Yelp API using the provided DataFrame and headers. 
    The DataFrame should contain 'latitude' and 'longitude' columns. 
    The function returns a new DataFrame with the extracted data.

    :param df1: DataFrame containing 'latitude' and 'longitude' columns
    :param headers: API headers for the Yelp API
    :returns: DataFrame with the extracted Yelp API data
    """
    yelp_data_extract = []

    # Iterating through first 100 station to pull Yelp data
    for _, row in df1[:10].iterrows():
        lat, lng = row['latitude'], row['longitude']
        url = f"https://api.yelp.com/v3/businesses/search?latitude={lat}&longitude={lng}"

        try:
            response = requests.get(url, headers=headers)
            response.raise_for_status()
        except requests.exceptions.RequestException as err:
            print(f"An error occurred: {err}")
            continue

        yelp_data = response.json()

        for result in yelp_data['businesses']:
            yelp_data_extract.append({
                "name" : result['name'],
                "rating" : result.get('rating'),
                "review_count" : result.get('review_count'),
                "price" : result.get('price'),
                "distance" : result.get('distance'),
                "category" : result.get('title'),
                "latitude" : lat,
                "longitude" : lng
            })

    df3 = pd.DataFrame(yelp_data_extract)

    return df3

# Call the function on the citybike dataframe to pull the Yelp API results
yelp_df = extract_yelp_data(df1, headers)
yelp_df.head()
yelp_df.to_csv('G:/Data/Python Project/yelp.csv')
